# Challenge 1 - T-test

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_re`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](https://researchbasics.education.uconn.edu/t-test/) and [this Quora](https://www.quora.com/What-is-the-difference-between-a-paired-and-unpaired-t-test). Make sure you understand when to use which type of t-test.

In [17]:
# Import libraries

import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest


from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem
from scipy.stats import ttest_rel


#### Import dataset

In this challenge we will work on the Pokemon dataset you have used last week. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.).

In [3]:
# Import dataset

pokemon = pd.read_csv('Pokemon.csv')

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


#### First we want to define a function with which we can test the means of a feature set of two samples.

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [12]:
def t_test_features(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    """Test means of a feature set of two samples

    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test

    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    results = {}

    for feature in features:
        results[feature] = ttest_ind(s1[feature], s2[feature], equal_var=False)[1]

    return results

#### Using the `t_test_features` function, conduct t-test for Lengendary vs non-Legendary pokemons.

*Hint: your output should look like below:*

```
{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}
 ```

In [13]:
# Your code here
legendary_pokemon = pokemon[pokemon['Legendary'] == True]
non_legendary_pokemon = pokemon[pokemon['Legendary'] == False]

features = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']

t_test_features(legendary_pokemon, non_legendary_pokemon, features)

{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [ ]:
# All p-values are way below 0.05, which means that we can reject the null hypothesis and affirm that both Legendary and non Legendary are significantly different

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [15]:
gen_1 = pokemon[pokemon['Generation'] == 1]
gen_2 = pokemon[pokemon['Generation'] == 2]

features = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']

t_test_features(gen_1, gen_2, features)


{'HP': 0.14551697834219623,
 'Attack': 0.24721958967217725,
 'Defense': 0.5677711011725426,
 'Sp. Atk': 0.12332165977104388,
 'Sp. Def': 0.18829872292645752,
 'Speed': 0.00239265937312135,
 'Total': 0.5631377907941676}

#### What conclusions can you make?

In [ ]:
# The p values for most of the stats are close to 1, which means that we ar enot confident about differences. The total is quite high, so we cannot reject the null hyphotesis.

#### Compare pokemons who have single type vs those having two types.

In [16]:
single = pokemon[pokemon['Type 2'].isna()]
dual = pokemon[pokemon['Type 2'].notna()]

features = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']

t_test_features(single, dual, features)


{'HP': 0.11314389855379413,
 'Attack': 0.00014932578145948305,
 'Defense': 2.7978540411514693e-08,
 'Sp. Atk': 0.00013876216585667907,
 'Sp. Def': 0.00010730610934512779,
 'Speed': 0.02421703281819093,
 'Total': 1.1157056505229961e-07}

#### What conclusions can you make?

In [ ]:
# We have mixed conclusions between some attributes. For Speed, Sp. Atk and Def, the values are significantly different. But we cannot reject the null hypothesis for all the rest, due to a p-value way above 0,5%.

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [19]:
t_stat_attack_defense, p_value_attack_defense = ttest_rel(pokemon['Attack'], pokemon['Defense'])

t_stat_attack_defense, p_value_attack_defense

(4.325566393330478, 1.7140303479358558e-05)

In [20]:
t_stat_spatk_spdef, p_value_spatk_spdef = ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])

t_stat_spatk_spdef, p_value_spatk_spdef

(0.853986188453353, 0.3933685997548122)

#### What conclusions can you make?

In [ ]:
# The p-values are very high. There's no significant difference